In [ ]:
!pip install tensorflow

In [ ]:
!pip install keras-tuner

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 10.8 MB/s eta 0:00:00


In [ ]:
!pip install xgboost

In [ ]:
!pip install statsmodels

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ost_data_clean.csv to ost_data_clean.csv


In [ ]:
import pandas as pd

df = pd.read_csv("ost_data_clean.csv", sep=';', engine='python')
df.head()


,Data,Ora,Shkembimi skeduluar nderkufitar,Shkembimi fizik nderkufitar,Frekuenca,Prodhimi Total,Ngarkesa Totale,AGC Integral of ACE,Shkembimi fizik nderkufitar AL-ME,Shkembimi skeduluar nderkufitar AL-ME,Shkembimi fizik nderkufitar AL-KS,Shkembimi skeduluar nderkufitar AL-KS,Shkembimi fizik nderkufitar AL-GR,Shkembimi skeduluar nderkufitar AL-GR
0,2024-02-23,H12,-24.0,-29.0,50.0,914.0,945.0,NaN,415.0,108.0,74.0,-11.0,-431.0,-121.0
1,2024-02-23,H13,-23.0,-22.0,50.0,945.0,985.0,NaN,473.0,20.0,84.0,-16.0,-501.0,-27.0
2,2024-02-23,H14,-23.0,-35.0,50.0,960.0,999.0,NaN,354.0,20.0,78.0,-16.0,-380.0,-27.0
3,2024-02-23,H15,-6.0,-44.0,49.0,951.0,990.0,NaN,58.0,-24.0,74.0,-17.0,-91.0,17.0
4,2024-02-23,H16,NaN,NaN,NaN,NaN,1016.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ============================================================
# Rolling-origin CV: XGB standalone + Hybrid (GRU + XGB residual)
# Google Colab friendly
# ============================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from xgboost import XGBRegressor

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

np.random.seed(42)
tf.random.set_seed(42)

# -------------------- Metrics --------------------
def eval_metrics(y_true_2d, y_pred_2d, tag=""):
    yt = y_true_2d.reshape(-1)
    yp = y_pred_2d.reshape(-1)

    mae = mean_absolute_error(yt, yp)
    rmse = np.sqrt(mean_squared_error(yt, yp))

    eps = 1e-6
    yt_safe = np.where(yt == 0, eps, yt)
    mape = mean_absolute_percentage_error(yt_safe, yp) * 100
    r2 = r2_score(yt, yp)

    print(f"[{tag}] MAE={mae:.2f} | RMSE={rmse:.2f} | MAPE={mape:.2f}% | R²={r2:.4f}")
    return mae, rmse, mape, r2

def summarize_cv(name, lists_dict):
    # lists_dict: {"MAE": [...], "RMSE": [...], ...}
    print(f"\n=== {name} – Rolling-origin CV Summary ===")
    for k, v in lists_dict.items():
        v = np.array(v, dtype=float)
        if k == "R2":
            print(f"{k:5s}: {v.mean():.4f} ± {v.std():.4f}")
        else:
            if k == "MAPE":
                print(f"{k:5s}: {v.mean():.2f}% ± {v.std():.2f}%")
            else:
                print(f"{k:5s}: {v.mean():.2f} ± {v.std():.2f}")

# -------------------- Data load & clean --------------------
df0 = pd.read_csv("ost_data_clean.csv", sep=';', engine='python')

df0['Data'] = pd.to_datetime(df0['Data'], errors='coerce')
df0['Ora'] = (
    df0['Ora'].astype(str)
    .str.extract(r'(\d+)', expand=False)
    .astype(int).astype(str).str.zfill(2)
)
df0['Datetime'] = pd.to_datetime(df0['Data'].dt.strftime('%Y-%m-%d') + ' ' + df0['Ora'],
                                 format='%Y-%m-%d %H', errors='coerce')
df0.set_index('Datetime', inplace=True)
df0.dropna(inplace=True)

# Total cross-border exchange if missing
if 'Shkembimi fizik nderkufitar' not in df0.columns:
    for c in ['Shkembimi fizik nderkufitar AL-ME',
              'Shkembimi fizik nderkufitar AL-KS',
              'Shkembimi fizik nderkufitar AL-GR']:
        if c not in df0.columns:
            df0[c] = 0.0
    df0['Shkembimi fizik nderkufitar'] = (
        df0['Shkembimi fizik nderkufitar AL-ME'] +
        df0['Shkembimi fizik nderkufitar AL-KS'] +
        df0['Shkembimi fizik nderkufitar AL-GR']
    )

base_cols = [
    'Ngarkesa Totale',
    'Prodhimi Total',
    'Shkembimi fizik nderkufitar',
    'Shkembimi fizik nderkufitar AL-ME',
    'Shkembimi fizik nderkufitar AL-KS',
    'Shkembimi fizik nderkufitar AL-GR'
]
df0 = df0[base_cols].copy()

# -------------------- Feature engineering --------------------
df = df0.copy()

df['Hour']      = df.index.hour
df['DayOfWeek'] = df.index.dayofweek
df['Month']     = df.index.month
df['Weekend']   = (df.index.dayofweek >= 5).astype(int)

target_col = 'Ngarkesa Totale'

# Lags (past only)
df['lag_1']   = df[target_col].shift(1)
df['lag_24']  = df[target_col].shift(24)
df['lag_48']  = df[target_col].shift(48)
df['lag_168'] = df[target_col].shift(168)

# Rolling (past window)
df['roll_mean_24'] = df[target_col].rolling(24).mean()
df['roll_std_24']  = df[target_col].rolling(24).std()
df['roll_max_24']  = df[target_col].rolling(24).max()
df['roll_min_24']  = df[target_col].rolling(24).min()

# Exchange dynamics
df['Total_Exchange'] = (
    df['Shkembimi fizik nderkufitar'] +
    df['Shkembimi fizik nderkufitar AL-ME'] +
    df['Shkembimi fizik nderkufitar AL-KS'] +
    df['Shkembimi fizik nderkufitar AL-GR']
)
df['Delta_Exchange'] = df['Total_Exchange'].diff()

def add_fourier(df_in, tag, period, K=3):
    t = np.arange(len(df_in), dtype=float)
    for k in range(1, K+1):
        df_in[f'{tag}_sin_{period}_{k}'] = np.sin(2*np.pi*k*t/period)
        df_in[f'{tag}_cos_{period}_{k}'] = np.cos(2*np.pi*k*t/period)
    return df_in

df = add_fourier(df, 'fourier_daily', 24,  K=3)
df = add_fourier(df, 'fourier_weekly', 168, K=3)

df.dropna(inplace=True)

# Columns for XGB (exclude target)
xgb_feat_cols = [c for c in df.columns if c != target_col]

# -------------------- Sequence makers --------------------
n_input  = 24
n_output = 24
t_idx = df.columns.get_loc(target_col)

def make_sequences(df_frame, n_in=24, n_out=24):
    """Return X (N,n_in,F), y (N,n_out), times (N,n_out)"""
    X, y, times = [], [], []
    idx = df_frame.index
    for i in range(n_in, len(df_frame) - n_out + 1):
        X.append(df_frame.iloc[i-n_in:i].values)
        y.append(df_frame.iloc[i:i+n_out, t_idx].values)
        times.append(idx[i:i+n_out].values)
    return np.array(X), np.array(y), np.array(times)

def generate_rolling_folds(df_frame, min_train_days=90, test_days=7):
    hours_per_day = 24
    min_train_len = min_train_days * hours_per_day
    test_len      = test_days * hours_per_day

    n = len(df_frame)
    folds = []
    train_start = 0
    train_end   = min_train_len

    while True:
        test_start = train_end
        test_end   = train_end + test_len
        if test_end > n:
            break
        folds.append((train_start, train_end, test_start, test_end))
        train_end = test_end  # expanding window
    return folds

def rows_to_features(times_2d, engineered_df, feat_cols):
    """(N,H) timestamps -> (N,H,F) tabular feature tensor from engineered_df."""
    N, H = times_2d.shape
    out = np.zeros((N, H, len(feat_cols)), dtype=np.float32)
    for i in range(N):
        rows = engineered_df.loc[times_2d[i]].reindex(columns=feat_cols).values
        out[i] = rows.astype(np.float32)
    return out

# -------------------- GRU builder --------------------
def build_gru(input_shape):
    model = Sequential([
        GRU(128, return_sequences=False,
            kernel_regularizer=l2(1e-4),
            recurrent_regularizer=l2(1e-4),
            bias_regularizer=l2(1e-4),
            input_shape=input_shape),
        Dropout(0.30),
        Dense(24, kernel_regularizer=l2(1e-4))
    ])
    model.compile(optimizer=Adam(learning_rate=5e-4), loss='mse')
    return model

# -------------------- Prepare fold data (GRU scaling only on train) --------------------
def prepare_fold_sequences(df_frame, target_col, tr_s, tr_e, te_s, te_e, n_in=24, n_out=24):
    df_train = df_frame.iloc[tr_s:tr_e].copy()
    df_test  = df_frame.iloc[te_s:te_e].copy()

    scaler = MinMaxScaler()
    scaler.fit(df_train)

    df_train_s = pd.DataFrame(scaler.transform(df_train), columns=df_frame.columns, index=df_train.index)
    df_test_s  = pd.DataFrame(scaler.transform(df_test),  columns=df_frame.columns, index=df_test.index)

    X_tr, y_tr, times_tr = make_sequences(df_train_s, n_in, n_out)
    X_te, y_te, times_te = make_sequences(df_test_s,  n_in, n_out)

    t_idx_local = df_frame.columns.get_loc(target_col)
    t_min = scaler.data_min_[t_idx_local]
    t_max = scaler.data_max_[t_idx_local]

    def inv_target(y_scaled):
        return y_scaled * (t_max - t_min) + t_min

    return X_tr, y_tr, times_tr, X_te, y_te, times_te, inv_target

# ============================================================
# Rolling-origin CV loop
# ============================================================
cv_folds = generate_rolling_folds(df, min_train_days=90, test_days=7)
print("Numri i folds:", len(cv_folds))

# Store fold metrics
xgb_metrics = {"MAE": [], "RMSE": [], "MAPE": [], "R2": []}
hyb_metrics = {"MAE": [], "RMSE": [], "MAPE": [], "R2": []}

# XGB params (fast & stable)
xgb_params = dict(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=1.0,
    min_child_weight=1,
    n_jobs=-1,
    random_state=42,
    tree_method="hist"
)

for fold_idx, (tr_s, tr_e, te_s, te_e) in enumerate(cv_folds, start=1):
    print(f"\n=== FOLD {fold_idx}/{len(cv_folds)} ===")
    print(f"Train: {tr_s}:{tr_e}, Test: {te_s}:{te_e}")

    # --- GRU data (scaled) ---
    X_tr, y_tr_s, times_tr_s, X_te, y_te_s, times_te_s, inv_target = prepare_fold_sequences(
        df, target_col, tr_s, tr_e, te_s, te_e, n_in=n_input, n_out=n_output
    )

    input_shape = (X_tr.shape[1], X_tr.shape[2])

    # Clear TF graph each fold (important for Colab RAM)
    tf.keras.backend.clear_session()

    gru = build_gru(input_shape)
    early = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    gru.fit(
        X_tr, y_tr_s,
        epochs=15,
        batch_size=64,
        validation_split=0.1,
        callbacks=[early],
        verbose=0
    )

    # GRU predictions (MW)
    y_tr_pred_s = gru.predict(X_tr, verbose=0)
    y_te_pred_s = gru.predict(X_te, verbose=0)

    y_tr_true = inv_target(y_tr_s)
    y_tr_pred = inv_target(y_tr_pred_s)

    y_te_true = inv_target(y_te_s)
    y_te_gru  = inv_target(y_te_pred_s)

    # --- Build tabular features for XGB from UN-SCALED engineered df ---
    # times_*_s are timestamps from scaled splits; they match original index, so OK.
    X_tab_tr = rows_to_features(times_tr_s, df, xgb_feat_cols)   # (Ntr,24,F)
    X_tab_te = rows_to_features(times_te_s, df, xgb_feat_cols)   # (Nte,24,F)

    Ntr, H, F = X_tab_tr.shape
    Nte = X_tab_te.shape[0]

    # ============================================================
    # (A) XGB standalone: y_hat directly
    # ============================================================
    y_te_xgb = np.zeros((Nte, H), dtype=np.float32)

    xgb_models_y = []
    for h in range(H):
        Xh_tr = X_tab_tr[:, h, :]
        yh_tr = y_tr_true[:, h]  # MW

        m = XGBRegressor(**xgb_params)
        m.fit(Xh_tr, yh_tr, verbose=False)
        xgb_models_y.append(m)

        y_te_xgb[:, h] = m.predict(X_tab_te[:, h, :]).astype(np.float32)

    print("XGB standalone fold metrics:")
    mae, rmse, mape, r2 = eval_metrics(y_te_true, y_te_xgb, f"XGB (fold {fold_idx})")
    xgb_metrics["MAE"].append(mae); xgb_metrics["RMSE"].append(rmse)
    xgb_metrics["MAPE"].append(mape); xgb_metrics["R2"].append(r2)

    # ============================================================
    # (B) Hybrid: GRU + XGB residual correction
    # ============================================================
    res_tr = (y_tr_true - y_tr_pred).astype(np.float32)  # (Ntr,24) MW residuals

    res_hat_te = np.zeros((Nte, H), dtype=np.float32)
    xgb_models_r = []
    for h in range(H):
        Xh_tr = X_tab_tr[:, h, :]
        rh_tr = res_tr[:, h]

        m = XGBRegressor(**{**xgb_params, "n_estimators": 400})
        m.fit(Xh_tr, rh_tr, verbose=False)
        xgb_models_r.append(m)

        res_hat_te[:, h] = m.predict(X_tab_te[:, h, :]).astype(np.float32)

    y_te_hybrid = (y_te_gru + res_hat_te).astype(np.float32)

    print("Hybrid (GRU + XGB residual) fold metrics:")
    mae, rmse, mape, r2 = eval_metrics(y_te_true, y_te_hybrid, f"Hybrid (fold {fold_idx})")
    hyb_metrics["MAE"].append(mae); hyb_metrics["RMSE"].append(rmse)
    hyb_metrics["MAPE"].append(mape); hyb_metrics["R2"].append(r2)

# ============================================================
# Summary
# ============================================================
summarize_cv("XGB standalone", xgb_metrics)
summarize_cv("Hybrid (GRU + XGB residual)", hyb_metrics)


Numri i folds: 54

=== FOLD 1/54 ===
Train: 0:2160, Test: 2160:2328


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


XGB standalone fold metrics:
[XGB (fold 1)] MAE=41.75 | RMSE=53.99 | MAPE=5.03% | R²=0.9093
Hybrid (GRU + XGB residual) fold metrics:
[Hybrid (fold 1)] MAE=58.41 | RMSE=73.86 | MAPE=7.35% | R²=0.8303

=== FOLD 2/54 ===
Train: 0:2328, Test: 2328:2496


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


XGB standalone fold metrics:
[XGB (fold 2)] MAE=50.94 | RMSE=65.35 | MAPE=5.40% | R²=0.8744
Hybrid (GRU + XGB residual) fold metrics:
[Hybrid (fold 2)] MAE=57.12 | RMSE=72.90 | MAPE=6.77% | R²=0.8437

=== FOLD 3/54 ===
Train: 0:2496, Test: 2496:2664


...

XGB standalone fold metrics:
[XGB (fold 52)] MAE=30.27 | RMSE=41.95 | MAPE=4.38% | R²=0.9241
Hybrid (GRU + XGB residual) fold metrics:
[Hybrid (fold 52)] MAE=41.87 | RMSE=55.63 | MAPE=6.04% | R²=0.8664

=== FOLD 53/54 ===
Train: 0:10896, Test: 10896:11064


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


XGB standalone fold metrics:
[XGB (fold 53)] MAE=31.38 | RMSE=37.31 | MAPE=4.38% | R²=0.9279
Hybrid (GRU + XGB residual) fold metrics:
[Hybrid (fold 53)] MAE=39.06 | RMSE=47.98 | MAPE=5.50% | R²=0.8807

=== FOLD 54/54 ===
Train: 0:11064, Test: 11064:11232


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


XGB standalone fold metrics:
[XGB (fold 54)] MAE=27.36 | RMSE=34.06 | MAPE=3.37% | R²=0.9626
Hybrid (GRU + XGB residual) fold metrics:
[Hybrid (fold 54)] MAE=42.19 | RMSE=51.53 | MAPE=5.29% | R²=0.9144

=== XGB standalone – Rolling-origin CV Summary ===
MAE  : 34.65 ± 10.36
RMSE : 44.94 ± 14.81
MAPE : 5.73% ± 5.11%
R2   : 0.9435 ± 0.0556

=== Hybrid (GRU + XGB residual) – Rolling-origin CV Summary ===
MAE  : 50.76 ± 18.39
RMSE : 64.38 ± 23.16
MAPE : 8.03% ± 5.60%
R2   : 0.8821 ± 0.1255


In [ ]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd

def prepare_fold_sequences(df_frame, target_col, train_start, train_end, test_start, test_end,
                           n_input=24, n_output=24):
    """
    Për një fold të dhënë, kthen:
    X_train, y_train, X_test, y_test, inv_target_fold
    duke bërë scaling vetëm mbi train.
    """
    # 1) Ndajmë train/test
    df_train = df_frame.iloc[train_start:train_end].copy()
    df_test  = df_frame.iloc[test_start:test_end].copy()

    # 2) Scale vetëm mbi TRAIN, pastaj e aplikojmë në train + test
    scaler = MinMaxScaler()
    scaler.fit(df_train)

    df_train_s = pd.DataFrame(
        scaler.transform(df_train),
        columns=df_frame.columns,
        index=df_train.index
    )

    df_test_s = pd.DataFrame(
        scaler.transform(df_test),
        columns=df_frame.columns,
        index=df_test.index
    )

    # 3) Krijojmë sequences për train/test me funksionin ekzistues
    X_train, y_train, _, _ = make_sequences(df_train_s, n_input, n_output)
    X_test,  y_test,  _, _ = make_sequences(df_test_s,  n_input, n_output)

    # 4) info mbi target-in
    t_idx = df_frame.columns.get_loc(target_col)
    t_min = scaler.data_min_[t_idx]
    t_max = scaler.data_max_[t_idx]

    def inv_target_fold(y_scaled):
        return y_scaled * (t_max - t_min) + t_min

    return X_train, y_train, X_test, y_test, inv_target_fold

# ================== Rolling-origin CV për GRU, Naive D-1 dhe HYBRID ==================

cv_folds = generate_rolling_folds(df, min_train_days=90, test_days=7)
print("Numri i folds:", len(cv_folds))

gru_mae_list, gru_rmse_list, gru_mape_list, gru_r2_list = [], [], [], []
naive_mae_list, naive_rmse_list, naive_mape_list, naive_r2_list = [], [], [], []

# <<< LISTAT PËR HYBRID >>>
hyb_mae_list, hyb_rmse_list, hyb_mape_list, hyb_r2_list = [], [], [], []

engineered_df = df  # përdorim df me të gjitha features (si te hybrid-i kryesor)

for fold_idx, (tr_s, tr_e, te_s, te_e) in enumerate(cv_folds):
    print(f"\n=== FOLD {fold_idx+1}/{len(cv_folds)} ===")
    print(f"Train: {tr_s}:{tr_e}, Test: {te_s}:{te_e}")

    # 1) Përgatisim sequences për këtë fold
    X_train, y_train, X_test, y_test, inv_target_fold = prepare_fold_sequences(
        df, target_col, tr_s, tr_e, te_s, te_e,
        n_input=n_input, n_output=n_output
    )

    input_shape_fold = (X_train.shape[1], X_train.shape[2])

    # 2) Ndërtojmë GRU për këtë fold
    def build_gru_fold(input_shape):
        model = Sequential([
            GRU(128, return_sequences=False,
                kernel_regularizer=l2(1e-4), recurrent_regularizer=l2(1e-4), bias_regularizer=l2(1e-4),
                input_shape=input_shape),
            Dropout(0.3),
            Dense(24, kernel_regularizer=l2(1e-4))
        ])
        model.compile(optimizer=Adam(learning_rate=5e-4), loss='mse')
        return model

    gru_fold = build_gru_fold(input_shape_fold)
    early_fold = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    history = gru_fold.fit(
        X_train, y_train,
        epochs=25,
        batch_size=64,
        validation_split=0.1,
        callbacks=[early_fold],
        verbose=0
    )

    # 3) Parashikimet (scaled -> MW)
    # --- TEST ---
    y_test_pred_s = gru_fold.predict(X_test, verbose=0)
    y_test_true   = inv_target_fold(y_test)
    y_test_pred   = inv_target_fold(y_test_pred_s)

    # --- TRAIN (DUHET PËR RESIDUALE HYBRID) ---
    y_train_pred_s = gru_fold.predict(X_train, verbose=0)
    y_train_true   = inv_target_fold(y_train)
    y_train_pred   = inv_target_fold(y_train_pred_s)

    # 4) Naive D-1 baseline për të njëjtin fold
    df_test_orig = df.iloc[te_s:te_e].copy()
    _, _, hours_test, times_test = make_sequences(df_test_orig, n_input, n_output)
    y_te_naive_D1 = make_naive_baseline(times_test, df, lag_hours=24)

    # 5) Metrikat GRU & Naive (siç i kishe)
    print("GRU fold metrics:")
    mae_g, rmse_g, mape_g, r2_g, _ = eval_metrics(y_test_true, y_test_pred, f"GRU (fold {fold_idx+1})")
    gru_mae_list.append(mae_g); gru_rmse_list.append(rmse_g)
    gru_mape_list.append(mape_g); gru_r2_list.append(r2_g)

    print("Naive D-1 fold metrics:")
    mae_n, rmse_n, mape_n, r2_n, _ = eval_metrics(y_test_true, y_te_naive_D1, f"Naive D-1 (fold {fold_idx+1})")
    naive_mae_list.append(mae_n); naive_rmse_list.append(rmse_n)
    naive_mape_list.append(mape_n); naive_r2_list.append(r2_n)

    # ================= HYBRID GRU + XGB PËR KËTË FOLD =================

    # Residualet në train: r = y_true - y_gru
    res_tr = y_train_true - y_train_pred           # (Ntr, 24)

    # Duam timestamps për TRAIN dhe TEST për të ndërtuar X_tab_tr, X_tab_te
    df_train_orig = df.iloc[tr_s:tr_e].copy()

    _, _, _, times_train_fold = make_sequences(df_train_orig, n_input, n_output)
    # times_test i kemi më sipër nga df_test_orig

    # rows_to_features dhe xgb_feat_cols i ke të deklaruara më sipër në notebook
    X_tab_tr = rows_to_features(times_train_fold)   # (Ntr, 24, F)
    X_tab_te = rows_to_features(times_test)         # (Nte, 24, F)

    Ntr_fold, H_fold, F_fold = X_tab_tr.shape

    xgb_models_fold = []
    for h in range(H_fold):
        Xh_tr = X_tab_tr[:, h, :]      # (Ntr, F)
        yh_tr = res_tr[:, h]           # (Ntr,)

        model_h = XGBRegressor(
            n_estimators=400,
            learning_rate=0.05,
            max_depth=6,
            subsample=0.8,
            colsample_bytree=0.8,
            reg_lambda=1.0,
            min_child_weight=1,
            n_jobs=-1,
            random_state=42
        )
        model_h.fit(Xh_tr, yh_tr)
        xgb_models_fold.append(model_h)

    # Parashikim i residualeve për TEST
    res_hat_te = np.zeros_like(y_test_pred)
    for h in range(H_fold):
        Xh_te = X_tab_te[:, h, :]
        res_hat_te[:, h] = xgb_models_fold[h].predict(Xh_te)

    # Hybrid = GRU + korrigjim XGB
    y_test_hybrid = y_test_pred + res_hat_te

    print("Hybrid fold metrics:")
    mae_h, rmse_h, mape_h, r2_h, _ = eval_metrics(y_test_true, y_test_hybrid, f"Hybrid (fold {fold_idx+1})")
    hyb_mae_list.append(mae_h); hyb_rmse_list.append(rmse_h)
    hyb_mape_list.append(mape_h); hyb_r2_list.append(r2_h)

# ================= Përmbledhje CV =================

import numpy as np

def summarize_cv(name, mae_list, rmse_list, mape_list, r2_list):
    print(f"\n=== {name} – Rolling-origin CV Summary ===")
    print(f"MAE  : {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
    print(f"RMSE : {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
    print(f"MAPE : {np.mean(mape_list):.2f}% ± {np.std(mape_list):.2f}%")
    print(f"R²   : {np.mean(r2_list):.4f} ± {np.std(r2_list):.4f}")

summarize_cv("GRU",         gru_mae_list,   gru_rmse_list,   gru_mape_list,   gru_r2_list)
summarize_cv("Naive D-1",   naive_mae_list, naive_rmse_list, naive_mape_list, naive_r2_list)
summarize_cv("Hybrid GRU+XGB", hyb_mae_list, hyb_rmse_list, hyb_mape_list, hyb_r2_list)



Numri i folds: 54

=== FOLD 1/54 ===
Train: 0:2160, Test: 2160:2328


ValueError: not enough values to unpack (expected 4, got 3)

In [ ]:
# ===================== Grafiku RMSE vs Fold =======================
# Numri i foldeve merret direkt nga gjatësia e listave të metrikeve
fold_numbers = list(range(1, len(gru_rmse_list) + 1))

plt.figure(figsize=(12, 5))
plt.plot(fold_numbers, gru_rmse_list,   marker='o', label='GRU')
plt.plot(fold_numbers, naive_rmse_list, marker='s', label='Naive D-1')
plt.plot(fold_numbers, hyb_rmse_list,   marker='d', label='Hybrid GRU+XGB')

plt.title('RMSE per Fold – Rolling-Origin Cross-Validation')
plt.xlabel('Fold')
plt.ylabel('RMSE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("rmse_vs_fold.png", dpi=600, bbox_inches='tight')
plt.show()

# ===================== Grafiku MAE vs Fold =======================
fold_numbers = list(range(1, len(gru_mae_list) + 1))

plt.figure(figsize=(12, 5))
plt.plot(fold_numbers, gru_mae_list,   marker='o', label='GRU')
plt.plot(fold_numbers, naive_mae_list, marker='s', label='Naive D-1')
plt.plot(fold_numbers, hyb_mae_list,   marker='d', label='Hybrid GRU+XGB')

plt.title('MAE per Fold – Rolling-Origin Cross-Validation')
plt.xlabel('Fold')
plt.ylabel('MAE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("mae_vs_fold.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
# ========================== Setup & Imports ==========================
import sys, subprocess, re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Ensure required packages
def _ensure(pkg):
    try:
        __import__(pkg)
    except ImportError:
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
_ensure("xgboost")
try:
    import holidays
    HAVE_HOLS = True
except Exception:
    HAVE_HOLS = False

import xgboost as xgb
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error,
    mean_absolute_percentage_error, r2_score
)

np.random.seed(42)

# ========================== 1) Load & clean ==========================
df = pd.read_csv("ost_data_clean.csv", sep=';', engine='python')
df.columns = df.columns.str.strip()

# Fix 'Ora' -> only digits, 2 chars
df['Ora'] = (
    df['Ora'].astype(str)
             .str.extract(r'(\d+)', expand=False)
             .fillna('0')
             .astype(int).astype(str).str.zfill(2)
)

# Parse date
df['Data'] = pd.to_datetime(df['Data'], errors='coerce')

# Handle possible '24' hour by rolling to next day at 00
mask_24 = df['Ora'] == '24'
df.loc[mask_24, 'Ora']  = '00'
df.loc[mask_24, 'Data'] = df.loc[mask_24, 'Data'] + pd.Timedelta(days=1)

# Build Datetime index
df['Datetime'] = pd.to_datetime(
    df['Data'].dt.strftime('%Y-%m-%d') + ' ' + df['Ora'],
    format='%Y-%m-%d %H', errors='coerce'
)
df.set_index('Datetime', inplace=True)
df.dropna(how='any', inplace=True)

# Standardize target name
if 'Ngarkesa Totale' in df.columns and 'Load' not in df.columns:
    df = df.rename(columns={'Ngarkesa Totale': 'Load'})

# Build total cross-border exchange if missing
if 'Shkembimi fizik nderkufitar' not in df.columns:
    for c in ['Shkembimi fizik nderkufitar AL-ME',
              'Shkembimi fizik nderkufitar AL-KS',
              'Shkembimi fizik nderkufitar AL-GR']:
        if c not in df.columns:
            df[c] = 0.0
    df['Shkembimi fizik nderkufitar'] = (
        df.get('Shkembimi fizik nderkufitar AL-ME', 0.0) +
        df.get('Shkembimi fizik nderkufitar AL-KS', 0.0) +
        df.get('Shkembimi fizik nderkufitar AL-GR', 0.0)
    )

# Keep a minimal base set if available
base_cols = ['Load', 'Prodhimi Total', 'Shkembimi fizik nderkufitar',
             'Shkembimi fizik nderkufitar AL-ME',
             'Shkembimi fizik nderkufitar AL-KS',
             'Shkembimi fizik nderkufitar AL-GR']
present = [c for c in base_cols if c in df.columns]
df = df[present].copy()

# Basic positivity filter for target if needed
df = df[df['Load'] > 0].copy()

# ========================== 2) Time features & holidays ==========================
df['hour']      = df.index.hour
df['dayofweek'] = df.index.dayofweek
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)
df['month']     = df.index.month

# country holidays (AL, ME, XK, GR, union ANY) — optional if holidays available
if HAVE_HOLS:
    years = sorted(df.index.year.unique())
    dser  = pd.Series(df.index.date, index=df.index)

    def _flag(code, fallback=None):
        try:
            cal = holidays.CountryHoliday(code, years=years)
            return dser.isin(cal).astype(int)
        except Exception:
            if fallback is not None:
                try:
                    cal = fallback(years=years)
                    return dser.isin(cal).astype(int)
                except Exception:
                    return pd.Series(0, index=df.index, dtype=int)
            return pd.Series(0, index=df.index, dtype=int)

    df['Holiday_AL'] = _flag('AL')
    df['Holiday_ME'] = _flag('ME')
    # Kosovo can be tricky in some versions
    try:
        df['Holiday_XK'] = _flag('XK', fallback=getattr(holidays, 'Kosovo', None))
    except Exception:
        df['Holiday_XK'] = 0
    df['Holiday_GR'] = _flag('GR')
    df['Holiday_ANY'] = df[['Holiday_AL','Holiday_ME','Holiday_XK','Holiday_GR']].max(axis=1).astype(int)
else:
    # If 'holidays' not available, just put zeros
    for c in ['Holiday_AL','Holiday_ME','Holiday_XK','Holiday_GR','Holiday_ANY']:
        df[c] = 0

# ========================== 3) Lags, rolling, dynamics, Fourier ==========================
df['lag_1']   = df['Load'].shift(1)
df['lag_24']  = df['Load'].shift(24)
df['lag_48']  = df['Load'].shift(48)
df['lag_168'] = df['Load'].shift(168)

df['roll_mean_24'] = df['Load'].rolling(24).mean()
df['roll_std_24']  = df['Load'].rolling(24).std()
df['roll_max_24']  = df['Load'].rolling(24).max()
df['roll_min_24']  = df['Load'].rolling(24).min()

# Exchange dynamics
ex_cols = [c for c in ['Shkembimi fizik nderkufitar',
                       'Shkembimi fizik nderkufitar AL-ME',
                       'Shkembimi fizik nderkufitar AL-KS',
                       'Shkembimi fizik nderkufitar AL-GR'] if c in df.columns]
df['Total_Exchange'] = df[ex_cols].sum(axis=1) if ex_cols else 0.0
df['Delta_Exchange'] = df['Total_Exchange'].diff()

# Fourier seasonality (daily & weekly)
def add_fourier(frame, tag, period, K=3):
    t = np.arange(len(frame), dtype=float)
    for k in range(1, K+1):
        frame[f'{tag}_sin_{period}_{k}'] = np.sin(2*np.pi*k*t/period)
        frame[f'{tag}_cos_{period}_{k}'] = np.cos(2*np.pi*k*t/period)
    return frame

df = add_fourier(df, 'fourier_daily', 24,  K=3)
df = add_fourier(df, 'fourier_weekly', 168, K=3)

# Drop NaNs created by lags/rolling
df.dropna(inplace=True)

# ========================== 4) Keep ONLY numeric columns ==========================
if 'Load' not in df.columns:
    raise ValueError("'Load' column not found after preprocessing.")

# Drop any non-numeric column (e.g., datetimes/strings)
non_numeric = df.select_dtypes(exclude=[np.number]).columns.tolist()
if non_numeric:
    df = df.drop(columns=non_numeric)

# Enforce Load numeric & valid
df['Load'] = pd.to_numeric(df['Load'], errors='coerce')
df = df.dropna(subset=['Load'])

# ========================== 5) Windowing 72 → 24 ==========================
def make_windows(frame, input_window=72, output_horizon=24, target='Load'):
    X, y, times = [], [], []
    cols = frame.columns.tolist()
    t_idx = cols.index(target)
    idx = frame.index

    for i in range(input_window, len(frame) - output_horizon):
        past   = frame.iloc[i-input_window:i].values        # (72, F)
        future = frame.iloc[i:i+output_horizon, t_idx].values  # (24,)
        X.append(past)
        y.append(future)
        times.append(idx[i:i+output_horizon].values)  # timestamps per horizon
    return np.array(X), np.array(y), np.array(times)

X_all, y_all, times_all = make_windows(df, 72, 24, target='Load')

# Cast to float32 for XGBoost compatibility
X_all = X_all.astype(np.float32)
y_all = y_all.astype(np.float32)

# Flatten the 3D past window into 2D features (N, 72*F)
N, Win, F = X_all.shape
X_all = X_all.reshape(N, Win*F)

# ========================== 6) Time split: 70/10/20 ==========================
n_total = X_all.shape[0]
i_train_end = int(n_total * 0.70)
i_val_end   = int(n_total * 0.80)

X_train, X_val, X_test = X_all[:i_train_end], X_all[i_train_end:i_val_end], X_all[i_val_end:]
y_train, y_val, y_test = y_all[:i_train_end], y_all[i_train_end:i_val_end], y_all[i_val_end:]
times_train, times_val, times_test = times_all[:i_train_end], times_all[i_train_end:i_val_end], times_all[i_val_end:]

# ========================== 7) XGBoost (one model per horizon) ==========================
params = {
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'learning_rate': 0.04,
    'max_depth': 6,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'lambda': 1.0,
    'min_child_weight': 1,
    'tree_method': 'hist',
    'seed': 42,
}
num_boost_round = 150
early_stopping_rounds = 10

models = []
eval_histories = []  # for optional plotting later

for h in range(y_train.shape[1]):  # 24 horizons
    dtrain = xgb.DMatrix(X_train, label=y_train[:, h])
    dval   = xgb.DMatrix(X_val,   label=y_val[:, h])

    evals_result = {}
    booster = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(dtrain, 'train'), (dval, 'val')],
        early_stopping_rounds=early_stopping_rounds,
        evals_result=evals_result,
        verbose_eval=False
    )
    models.append(booster)
    eval_histories.append(evals_result)

# ========================== 8) Predict on TEST ==========================
dtest = xgb.DMatrix(X_test)
y_pred = np.column_stack([m.predict(dtest, iteration_range=(0, m.best_iteration+1))
                          for m in models])

# ========================== 9) Metrics ==========================
def eval_metrics(y_true_2d, y_pred_2d, tag="XGB 72→24"):
    yt = y_true_2d.reshape(-1)
    yp = y_pred_2d.reshape(-1)

    mae = mean_absolute_error(yt, yp)

    # Ketu e ndryshojme: llogarisim RMSE pa 'squared=False'
    mse  = mean_squared_error(yt, yp)  # vetem MSE
    rmse = np.sqrt(mse)               # RMSE = sqrt(MSE)

    eps = 1e-6
    yt_safe = np.where(yt == 0, eps, yt)
    mape = mean_absolute_percentage_error(yt_safe, yp) * 100
    r2   = r2_score(yt, yp)
    acc  = 100 - mape

    print(f"[{tag}] MAE: {mae:.2f} | RMSE: {rmse:.2f} | MAPE: {mape:.2f}% | R²: {r2:.4f} | Accuracy: {acc:.2f}%")
    return mae, rmse, mape, r2, acc


print("\n--- Metrics on TEST ---")
eval_metrics(y_test, y_pred, tag="XGB (numeric-only, 72→24, holidays+lags+fourier)")

# ========================== 10) Plot first test sample ==========================
plt.figure(figsize=(12,5))
plt.plot(y_test[0], label='Actual')
plt.plot(y_pred[0], label='Prediction')
plt.title('24-hour Forecast (First Test Sample) — XGBoost')
plt.xlabel('Hour ahead'); plt.ylabel('Load (MW)')
plt.grid(True); plt.legend()
plt.savefig("xgb_forecast_first_sample.png", dpi=600, bbox_inches='tight')
plt.show()

# Average validation loss across horizons

try:
    min_len = min(len(hist['val']['rmse']) for hist in eval_histories)
    avg_val = np.mean([hist['val']['rmse'][:min_len] for hist in eval_histories], axis=0)
    avg_tr  = np.mean([hist['train']['rmse'][:min_len] for hist in eval_histories], axis=0)
    plt.figure(figsize=(10,4))
    plt.plot(avg_tr, label='Train RMSE (avg over 24 models)')
    plt.plot(avg_val, label='Val RMSE (avg over 24 models)')
    plt.title('Average RMSE over horizons during training (XGBoost)')
    plt.xlabel('Boosting round'); plt.ylabel('RMSE'); plt.grid(True); plt.legend()
    plt.savefig("xgb_avg_rmse_train_val.png", dpi=600, bbox_inches='tight')
    plt.show()

except Exception:
    pass


In [ ]:
# ========================== 11) Rolling-origin Cross-Validation for XGB ==========================

def generate_rolling_folds_N(N, min_train_days=90, test_days=7):
    """
    Gjeneron folds me dritare kohore në nivel mostre (N = numri i dritareve 72→24).
    Çdo mostër korrespondon me një orë, pasi dritaret rrëshqasin me hap 1.
    """
    hours_per_day = 24
    min_train_len = min_train_days * hours_per_day
    test_len      = test_days * hours_per_day

    folds = []
    train_start = 0
    train_end   = min_train_len

    while True:
        test_start = train_end
        test_end   = train_end + test_len

        if test_end > N:
            break

        folds.append((train_start, train_end, test_start, test_end))
        # expanding window: trajnimi zgjerohet deri në fund të testit
        train_end = test_end

    return folds

# Gjenerojmë folds në bazë të numrit total të mostrave (X_all)
N_all = X_all.shape[0]
cv_folds_xgb = generate_rolling_folds_N(N_all, min_train_days=90, test_days=7)
print("Numri i folds (XGB):", len(cv_folds_xgb))

xgb_mae_list, xgb_rmse_list, xgb_mape_list, xgb_r2_list = [], [], [], []

for fold_idx, (tr_s, tr_e, te_s, te_e) in enumerate(cv_folds_xgb):
    print(f"\n=== XGB FOLD {fold_idx+1}/{len(cv_folds_xgb)} ===")
    print(f"Train: {tr_s}:{tr_e}, Test: {te_s}:{te_e}")

    # 1) Ndajmë train/test për këtë fold (72→24 dritare tashmë të ndërtuara)
    X_tr_fold = X_all[tr_s:tr_e]
    y_tr_fold = y_all[tr_s:tr_e]
    X_te_fold = X_all[te_s:te_e]
    y_te_fold = y_all[te_s:te_e]

    # 2) Brenda train-it të këtij fold-i ndajmë edhe një VAL të vogël (p.sh. 10%)
    Ntr_fold = X_tr_fold.shape[0]
    if Ntr_fold < 100:
        # nëse kemi shumë pak mostra, përdorim të gjithë train-in pa VAL të brendshëm
        X_tr_sub, y_tr_sub = X_tr_fold, y_tr_fold
        X_val_sub, y_val_sub = None, None
    else:
        split_idx = int(Ntr_fold * 0.9)
        X_tr_sub,  y_tr_sub  = X_tr_fold[:split_idx],  y_tr_fold[:split_idx]
        X_val_sub, y_val_sub = X_tr_fold[split_idx:], y_tr_fold[split_idx:]

    # 3) Trajnojmë 24 modele XGB (një për çdo horizont)
    models_fold = []
    for h in range(y_tr_fold.shape[1]):  # 24 horizons
        dtrain = xgb.DMatrix(X_tr_sub, label=y_tr_sub[:, h])

        if X_val_sub is not None:
            dval   = xgb.DMatrix(X_val_sub, label=y_val_sub[:, h])
            booster = xgb.train(
                params,
                dtrain,
                num_boost_round=num_boost_round,
                evals=[(dtrain, 'train'), (dval, 'val')],
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=False
            )
        else:
            # pa val të brendshëm – trenim i thjeshtë pa early stopping
            booster = xgb.train(
                params,
                dtrain,
                num_boost_round=num_boost_round,
                evals=[(dtrain, 'train')],
                verbose_eval=False
            )

        models_fold.append(booster)

    # 4) Parashikime në TEST për këtë fold
    dtest_fold = xgb.DMatrix(X_te_fold)
    y_pred_fold = np.column_stack([
        m.predict(dtest_fold, iteration_range=(0, m.best_iteration+1)
                  if hasattr(m, "best_iteration") and m.best_iteration is not None
                  else (0, num_boost_round))
        for m in models_fold
    ])

    # 5) Metrikat për këtë fold
    mae_f, rmse_f, mape_f, r2_f, _ = eval_metrics(y_te_fold, y_pred_fold,
                                                  tag=f"XGB 72→24 (fold {fold_idx+1})")
    xgb_mae_list.append(mae_f)
    xgb_rmse_list.append(rmse_f)
    xgb_mape_list.append(mape_f)
    xgb_r2_list.append(r2_f)

# ========================== 12) Përmbledhje e CV për XGB ==========================

def summarize_cv(name, mae_list, rmse_list, mape_list, r2_list):
    print(f"\n=== {name} – Rolling-origin CV Summary ===")
    print(f"MAE  : {np.mean(mae_list):.2f} ± {np.std(mae_list):.2f}")
    print(f"RMSE : {np.mean(rmse_list):.2f} ± {np.std(rmse_list):.2f}")
    print(f"MAPE : {np.mean(mape_list):.2f}% ± {np.std(mape_list):.2f}%")
    print(f"R²   : {np.mean(r2_list):.4f} ± {np.std(r2_list):.4f}")

summarize_cv("XGB 72→24", xgb_mae_list, xgb_rmse_list, xgb_mape_list, xgb_r2_list)

# ========================== 13) Grafiku RMSE vs Fold & MAE vs Fold (XGB) ==========================

fold_numbers = list(range(1, len(xgb_rmse_list) + 1))

plt.figure(figsize=(12, 5))
plt.plot(fold_numbers, xgb_rmse_list, marker='o', label='XGB 72→24')
plt.title('RMSE per Fold – Rolling-Origin Cross-Validation (XGB)')
plt.xlabel('Fold')
plt.ylabel('RMSE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("xgb_rmse_vs_fold.png", dpi=600, bbox_inches='tight')
plt.show()

plt.figure(figsize=(12, 5))
plt.plot(fold_numbers, xgb_mae_list, marker='s', label='XGB 72→24')
plt.title('MAE per Fold – Rolling-Origin Cross-Validation (XGB)')
plt.xlabel('Fold')
plt.ylabel('MAE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("xgb_mae_vs_fold.png", dpi=600, bbox_inches='tight')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Sigurohemi që të gjithë listat kanë të paktën kaq folds
n_folds = min(
    len(gru_rmse_list),
    len(naive_rmse_list),
    len(hyb_rmse_list),
    len(xgb_rmse_list)
)

folds = np.arange(1, n_folds + 1)

# ===================== RMSE per Fold – TË GJITHË MODELET =====================
plt.figure(figsize=(12, 5))
plt.plot(folds, naive_rmse_list[:n_folds], marker='o', label='Naive D-1')
plt.plot(folds, gru_rmse_list[:n_folds],   marker='s', label='GRU')
plt.plot(folds, xgb_rmse_list[:n_folds],   marker='^', label='XGB 72→24')
plt.plot(folds, hyb_rmse_list[:n_folds],   marker='d', label='Hybrid GRU+XGB')

plt.title('RMSE per Fold – Rolling-Origin Cross-Validation (Naive vs GRU vs XGB vs Hybrid)')
plt.xlabel('Fold')
plt.ylabel('RMSE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("rmse_vs_fold_all_models.png", dpi=600, bbox_inches='tight')
plt.show()

# ===================== MAE per Fold – TË GJITHË MODELET =====================
plt.figure(figsize=(12, 5))
plt.plot(folds, naive_mae_list[:n_folds], marker='o', label='Naive D-1')
plt.plot(folds, gru_mae_list[:n_folds],   marker='s', label='GRU')
plt.plot(folds, xgb_mae_list[:n_folds],   marker='^', label='XGB 72→24')
plt.plot(folds, hyb_mae_list[:n_folds],   marker='d', label='Hybrid GRU+XGB')

plt.title('MAE per Fold – Rolling-Origin Cross-Validation (Naive vs GRU vs XGB vs Hybrid)')
plt.xlabel('Fold')
plt.ylabel('MAE')
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("mae_vs_fold_all_models.png", dpi=600, bbox_inches='tight')
plt.show()
